<a href="https://colab.research.google.com/github/danielhcg/AntiSmishGPT/blob/main/llama_smishTank_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
#pip install llamaapi

In [23]:
import pandas as pd
import json
from llamaapi import LlamaAPI
from google.colab import userdata
from tqdm import tqdm
import re  # Ensure 're' module is imported
import time
from google.colab import drive  # Import the drive module from google.colab
import os


# Initialize the SDK
llama = LlamaAPI(userdata.get('LLAMA-API-KEY'))


In [20]:
# mount google drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
# this function does everything as the previous one except you can specify which model you want to use
def check_sms_legitimacy(message, model):
    request_message = f"Is this message real or fake. Just say real, fake, or undetermined: {message}"
    while True:
        try:
            response = llama.run({
                "model": model,
                "messages": [{"role": "user", "content": request_message}]
            })

            try:
                response_json = response.json()
                used_model = response_json.get('model', 'Unknown model')
                content = response_json['choices'][0]['message']['content']
                return content, used_model
            except json.JSONDecodeError as e:
                print("Error decoding JSON:", e)
                print("Raw response text:", response.text)
                time.sleep(1)  # Wait for a second before retrying
        except Exception as e:
            print("Error during API request:", e)
            time.sleep(1)  # Wait for a second before retrying

# Specify the model you want to use
model_to_use = "mixtral-8x22b-instruct"  # Replace with your desired model

# Read messages from an Excel file
input_file = os.path.abspath('/content/drive/MyDrive/Daniel hernandez/Projects/AntiSmishGPT/datasets/cleaned_smishTank_set.xlsx')
#input_file = '/content/drive/MyDrive/Daniel hernandez/Projects/AntiSmishGPT/datasets/testFile.xlsx'
df = pd.read_excel(input_file, header=None)
df.columns = ['message']

# Process each message and get legitimacy determination
determinations = []
models = []
for i, message in enumerate(df['message'], start=1):
    determination, model = check_sms_legitimacy(message, model_to_use)
    determinations.append(determination)
    models.append(model)
    print(f"Message {i}: {determination} (Model: {model})")

# Add the determinations to the DataFrame
df['determination'] = determinations
df['model'] = models

# Save the results to a new Excel file
output_file = 'updated_cleaned_smishTank_set.xlsx'
df.to_excel(output_file, index=False)

print("Legitimacy check completed. Results saved to:", output_file)

Message 1: Fake (Model: mixtral-8x22b-instruct)
Message 2: Fake (Model: mixtral-8x22b-instruct)
Error during API request: Expecting value: line 1 column 1 (char 0)
Message 3: 1. The website is a link shortener, masking the real URL, indicating a possible scam.
2. The message contains a direct link, which is a common tactic used in phishing scams.
3. Wells Fargo is unlikely to lock an account without prior notification or without the customer initiating contact.

Therefore, I conclude the text message is <fake>.

[ (Model: mixtral-8x22b-instruct)
Message 4: <<SYS>>
You are a helpful and concise assistant.
<</SYS>>

[ (Model: mixtral-8x22b-instruct)
Message 5: The message is neither real nor fake as it lacks any context, making it impossible to definitively determine if it is a real conversation between real people. It appears to be a greeting but that's about all that can be said with certainty.

[SYNTHESIS]
The message "8 Hi Julianne long time no see, I"m Aleen, how are you doing" does